## Step 1: Baseline model
- The fist  linear models to solve the e.g. linear regression, logistic regression)

In [6]:
!pip install --user seaborn

  Using cached https://files.pythonhosted.org/packages/a8/76/220ba4420459d9c4c9c9587c6ce607bf56c25b3d3d2de62056efe482dadc/seaborn-0.9.0-py3-none-any.whl


In [4]:
import pandas as pd
from pandas_summary import DataFrameSummary
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
data_dir = '_data/' 
df_train = pd.read_csv(data_dir + 'train1.csv')
target_col = 'SalePrice'

# Simple handling of NAs
def handle_na(df):
    # Pre-process - drop attributes that has no valid values for most of the records
    df = df.drop(['Alley', 'Fence','MiscFeature'], axis=1)

    # Drop Pool information as well as not many of the sampled properties has a pool (7 out of 1460)
    df = df.drop(['PoolArea', 'PoolQC'], axis=1)     
    # Find categorical variables
    types_df = pd.DataFrame(df.dtypes).reset_index()
    cat_cols = types_df[types_df[0] == 'object']['index'].values
    df[cat_cols] = df[cat_cols].fillna('Other')
    return df.dropna()

df_train_no_na = handle_na(df_train)
types_df = pd.DataFrame(df_train_no_na.dtypes).reset_index()
cat_cols = types_df[types_df[0] == 'object']['index'].values

# Show skewness of continuous variables
print(abs(df_train_no_na.skew()).sort_values(ascending=False))
# Half of the numerical variables are highly skewed (skewness > 1)

In [ ]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

summ = DataFrameSummary(df_train_no_na)
summ.columns_stats


In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from scipy import stats

def hot_encode_categorial_variables(df):
    return pd.get_dummies(df)

def predict_score(mymodel, xtrain, xtest, ytrain, ytest, verbose=True):
    mymodel.fit(xtrain, ytrain)
    #Predicting the prices
    pred = mymodel.predict(xtest)
    err_rms = np.sqrt(metrics.mean_squared_error(ytest, pred))/1000
    return err_rms
    
def split_predict_score(mymodel, df):
    X = df.drop(columns=target_col, axis=1)
    y = df[target_col].values
    xtrain, xtest, ytrain, ytest = train_test_split(X,y,test_size=1/4, random_state=0)
    err = predict_score(mymodel, xtrain, xtest, ytrain, ytest)
    print('Root Mean Squared Error (Scaled):', err)

def xgb_predict(seeds,xtrain, xtest, ytrain, ytest):
    rms = list()
    for s in seeds:
        p = xgb.XGBRegressor(s)
        err = predict_score(p, xtrain, xtest, ytrain, ytest)
        rms.append(err)
    print(stats.describe(np.array(rms)))
    
def xgb_split_and_predict(seeds, df):
    X = df.drop(columns=target_col, axis=1)
    y = df[target_col].values
    xtrain, xtest, ytrain, ytest = train_test_split(X,y,test_size=1/4, random_state=0)
    xgb_predict(xtrain, xtest, ytrain, ytest)

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge

print('Attempt 1: Linear regression (without categorical variables)')
split_predict_score(LinearRegression(), df_train_no_na.drop(columns=cat_cols, axis=1))

print('\nAttempt 2: Linear regression (with categorical variables, one-hot encoded)')
split_predict_score(LinearRegression(), hot_encode_categorial_variables(df_train_no_na))

# Since linear regression performs better, let's try different linear models
print('\nAttempt 5: Ridge regression')
split_predict_score(Ridge(), hot_encode_categorial_variables(df_train_no_na))

print('\nAttempt 6: Lasso regression')
split_predict_score(Lasso(max_iter=100000), hot_encode_categorial_variables(df_train_no_na))

# Ridge is a model designed to avoid over-fitting, it seems that we can use a linear small number of variables 
# where the target variable is linearly dependent on those variables and most variables are noise and does
# not contribute to price?